<a href="https://colab.research.google.com/github/hameedolaniyi97/Classifying-Musical-Instruments-with-Transfer-Learning/blob/main/musical_instrument_Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, optim
import numpy as np
import timm
import wandb
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nikolasgegenava/music-instruments")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/music-instruments


In [ ]:
data_path = os.path.join(path, 'music_instruments')

In [ ]:
# setup the transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.ColorJitter(brightness=0.2, saturation=0.2,
                           hue=0.1,contrast=0.1),
    transforms.Resize((200,200))
])

In [ ]:
# load the dataset
data = datasets.ImageFolder(root=data_path, transform=transform)

In [ ]:
# split the data into train and test set
train_size = int(0.6 * len(data))
test_size = len(data) - train_size
train_data, test_data = random_split(dataset=data, lengths=[train_size, test_size])

# repeat this step to split test (40% of the data) into train and test set to fit gpu.
train_size = int(0.8 * len(test_data))
test_size = len(test_data) - train_size
train_data, test_data = random_split(dataset=test_data, lengths=[train_size, test_size])


# setup the data loader
BATCH_SIZE = 32
train_loader = DataLoader(dataset = train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# setup the model architecture and hyperparameters

model = timm.create_model(model_name='efficientnet_b3', pretrained=True)
model.classifier = nn.Linear(in_features=model.classifier.in_features, out_features=10)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
LEARNING_RATE = 1e-4
optimizer = optim.AdamW(params=model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=2, gamma=0.1)
EPOCHS = 10
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

In [ ]:
music_ins_list = ['accordion','banjo','drum',
               'flute','guitar','harmonica',
               'saxophone','sitar','tabla','violin']
len(music_ins_list)

10

In [ ]:
# init wandb for tracking

run = wandb.init(
    project = 'music_instrument Classification With Fine Tuned EfficientNet-B3',
    name= 'second run',
    config= {
        'model_name': 'efficientnet-b3',
        'device': device,
        'data_path': data_path,
        'learning rate': LEARNING_RATE,
        'optimizer': 'AdamW',
        'weight decay': 0.01,
        'scheduler': {'step': 2, 'gamma': 0.1},
        'epochs': EPOCHS
    }
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: datawhizcoder (datawhizcoder-apple) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# begin the training loop


for epoch in range(EPOCHS):
  train_epoch_loss, test_epoch_loss = [], []
  train_epoch_labels, train_epoch_preds = [], []
  test_epoch_labels, test_epoch_preds = [], []
  train_batch = tqdm(train_loader, desc= f'Training Epoch: {epoch+1}/{EPOCHS}')
  for image, label in train_batch:
    model.train()
    image, label = image.to(device), label.to(device)
    train_preds = model(image)
    loss = criterion(train_preds, label)

    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_epoch_loss.append(loss.item())
    train_epoch_labels.extend(label.cpu().detach().numpy())
    _, train_preds = torch.max(train_preds, 1)
    train_epoch_preds.extend(train_preds.cpu().detach().numpy())

    train_batch.set_postfix(loss=loss.item())

  scheduler.step()
  # compute the f1_score, precision and recall for training set
  train_final_loss = sum(train_epoch_loss)/len(train_epoch_loss)
  train_f1 = f1_score(train_epoch_labels, train_epoch_preds, average='weighted')
  train_precision = precision_score(train_epoch_labels, train_epoch_preds, average='weighted')
  train_recall = recall_score(train_epoch_labels, train_epoch_preds, average='weighted')

  # now evaluate
  model.eval()
  with torch.no_grad():
    test_batch = tqdm(test_loader, desc= f'Evaluating tests for Epoch: {epoch+1}/{EPOCHS}')
    for test_image, test_labels in test_batch:
      test_image, test_labels = test_image.to(device), test_labels.to(device)
      test_preds = model(test_image)


      test_loss = criterion(test_preds, test_labels)
      test_epoch_loss.append(test_loss.item())
      test_epoch_labels.extend(test_labels.cpu().detach().numpy())
      _, test_preds = torch.max(test_preds, 1)
      test_epoch_preds.extend(test_preds.cpu().detach().numpy())

  # compute the f1_score, precision and recall for test set
  test_final_loss = sum(test_epoch_loss)/len(test_epoch_loss)
  test_f1 = f1_score(test_epoch_labels, test_epoch_preds, average='weighted')
  test_precision = precision_score(test_epoch_labels, test_epoch_preds, average='weighted')
  test_recall = recall_score(test_epoch_labels, test_epoch_preds, average='weighted')

  print(f'Completed training Epoch: {epoch+1}..f1: {train_f1, test_f1}, loss: {train_final_loss, test_final_loss}')
  print(f'Completed training Epoch: {epoch+1}..recall: {train_recall, test_recall}, precision: {train_precision, test_precision}')

  # append result in wandb
  run.log({
      'epochs': epoch + 1,
      'train loss': train_final_loss,
      'test loss': test_final_loss,
      'train f1': train_f1,
      'train precision': train_precision,
      'train recall': train_recall,
      'test f1': test_f1,
      'test precision': test_precision,
      'test recall': test_recall
  })

run.finish()


Evaluating tests for Epoch: 1/10: 100%|██████████| 5/5 [00:02<00:00,  2.50it/s]


Completed training Epoch: 1..f1: (0.3141418324432297, 0.6284903071997995), loss: (2.1649285617627596, 1.9268031358718871)
Completed training Epoch: 1..recall: (0.3494809688581315, 0.6413793103448275), precision: (0.4032133603720137, 0.6750130419501617)


Evaluating tests for Epoch: 2/10: 100%|██████████| 5/5 [00:01<00:00,  4.49it/s]


Completed training Epoch: 2..f1: (0.7753826445209852, 0.7940261221414876), loss: (1.681069581132186, 1.4647547721862793)
Completed training Epoch: 2..recall: (0.7785467128027682, 0.7931034482758621), precision: (0.7910715781703527, 0.8112777840820437)


Evaluating tests for Epoch: 3/10: 100%|██████████| 5/5 [00:01<00:00,  4.72it/s]


...

Evaluating tests for Epoch: 9/10: 100%|██████████| 5/5 [00:01<00:00,  4.57it/s]


Completed training Epoch: 9..f1: (0.8660739241992864, 0.789635730612585), loss: (1.248556181004173, 1.423336958885193)
Completed training Epoch: 9..recall: (0.8685121107266436, 0.7931034482758621), precision: (0.8723891312418691, 0.8017264429282684)


Evaluating tests for Epoch: 10/10: 100%|██████████| 5/5 [00:01<00:00,  3.95it/s]

Completed training Epoch: 10..f1: (0.8778881960673635, 0.7797370739124863), loss: (1.2394441303453947, 1.404770851135254)
Completed training Epoch: 10..recall: (0.8788927335640139, 0.7793103448275862), precision: (0.8833300262680382, 0.8007193752832699)


epochs,▁▂▃▃▄▅▆▆▇█
test f1,▁█▆███▇▇█▇
test loss,█▂▂▁▁▁▂▁▁▁
test precision,▁▇▆▇▇█▆▆▇▆
test recall,▁█▆███▇▆█▇
train f1,▁▇████████
train loss,█▄▂▁▁▁▁▁▁▁
train precision,▁▇████████
train recall,▁▇▇███████
epochs,10
test f1,0.77974


In [ ]:
from PIL import Image

softmax = nn.Softmax()


def predict_music_ins() -> None:
  new_image = Image.open(input('Enter image url: ')).convert('RGB')
  new_image = transform(new_image).unsqueeze(0)
  new_image = new_image.to(device)
  preds = model(new_image)
  preds = softmax(preds)
  probability, prediction = torch.max(preds, 1)

  print(f'This is an image of {music_ins_list[prediction.item()]} with probability of {probability.item()}')




predict_music_ins()

Enter image url: /content/OIP (1).webp
This is an image of tabla with probability of 0.17019538581371307


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
